In [ ]:
import pandas as pd
import numpy as np
import datetime
import requests
import threading
import time
import concurrent
import os, random
import csv
import glob

from ta import add_all_ta_features
from ta.utils import dropna
from ta.trend import STCIndicator

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
tickers = []
with open('symbols.csv', newline='') as inputfile:
    for row in csv.reader(inputfile):
        tickers.append(row[0])
    tickers.pop(0) #remove zero index

In [ ]:
def fetch_ohlc(symbol, timeframe, timesymbol):

        symbol = symbol
        timeframe = timeframe
        timesymbol = timesymbol

        # fetch data-binance api
        candlestick_url = (
            "https://fapi.binance.com/fapi/v1/continuousKlines?pair="
            + symbol
            + "&contractType=PERPETUAL&interval="
            + str(timeframe)
            + timesymbol
            + "&limit=1500"
        )
        candlestick_chart = requests.get(candlestick_url).json()
        candlestick_df = pd.DataFrame(candlestick_chart)
        candlestick_df = candlestick_df.iloc[:, 1:7]
        candlestick_df.columns = ["open", "high", "low", "close", "volume", "date"]
        candlestick_df["date"] = pd.to_datetime(
            candlestick_df["date"], unit="ms"
        ).round("1s")
        candlestick_df.insert(0, "date", candlestick_df.pop("date"))

        # reset to midnight
        candlestick_df.date = pd.to_datetime(candlestick_df.date)
        min_date = candlestick_df.date.min()
        NextDay_Date = (min_date + datetime.timedelta(days=1)).replace(
            hour=0, minute=0, second=0, microsecond=0
        )
        candlestick_df = candlestick_df[candlestick_df.date >= NextDay_Date].copy()

        candlestick_df = candlestick_df.set_index("date")
        candlestick_df["symbol"] = symbol

        ohlc_data = candlestick_df

        cols = ["open", "high", "low", "close", "volume"]
        ohlc_data[cols] = ohlc_data[cols].apply(pd.to_numeric, errors="coerce")
        ohlc_data[cols] = ohlc_data[cols].round(decimals=8) #decimal important for small alts
        return ohlc_data

In [ ]:
def ohlc_30(symbol):
    try:
        ohlc_30 = fetch_ohlc(symbol,30,'m')
        ohlc_30.to_csv(f"OHLC30m/"+symbol+"30m.csv")
    except ValueError:
        pass

In [ ]:
with concurrent.futures.ProcessPoolExecutor() as executor:
     result = [executor.submit(ohlc_30,symbol) 
               for symbol in tickers]

In [ ]:
def compute_IB(symbol):
    try:

        ohlc_smaller = pd.read_csv(f"OHLC30m/"+symbol+'30m.csv')
        ohlc_smaller['date'] = pd.to_datetime(ohlc_smaller['date'])
        ohlc_smaller = ohlc_smaller.set_index('date')

        daily_ohlc_data = ohlc_smaller.resample('D').apply(
                {
                    'open':'first',
                    'high':'max',
                    'low' : 'min',
                    'close': 'last'
                }
            ).dropna()

        initial_balance = (ohlc_smaller[ohlc_smaller.index.time < datetime.time(1,00)].resample('D')
                               .apply({'high':'max','low':'min'}).dropna())

        initial_balance['symbol'] = symbol

        column_to_move = initial_balance.pop('symbol')
        initial_balance.insert(0, 'symbol', column_to_move)

        initial_balance['width'] = initial_balance['high'].astype(float) - initial_balance['low'].astype(float)
        initial_balance['mid'] =  (initial_balance['high'].astype(float) + initial_balance['low'].astype(float))/2

        #range open

        initial_balance['range_open'] = daily_ohlc_data['close'].shift(+1) < initial_balance['mid']
        initial_balance['range_open'] = initial_balance['range_open'].apply(lambda x: 'bottom' if x == True else 'top')

        #opportunity(based on range open) !not signal
        initial_balance['ib_opportunity'] = initial_balance.apply(lambda x: 'short' if x.range_open == 'top' else 'long', axis=1)

        #set up IB width

        window =21 
        ib_avg_width = initial_balance['width'].rolling(window=window).mean()
        ib_sd = initial_balance['width'].rolling(window=window).std()

        initial_balance['narrow_ib'] =initial_balance['width'] < ib_avg_width - ib_sd
        initial_balance['moderate_ib'] = (initial_balance['width']>= ib_avg_width-ib_sd) & (initial_balance['width']<= ib_avg_width+ib_sd)
        initial_balance['wide_ib'] =initial_balance['width'] > ib_avg_width + ib_sd

        #IB width summary

        initial_balance['width_status'] = initial_balance.apply(lambda x: 'narrow' if x.narrow_ib == True else ( 'moderate' if x.moderate_ib == True else 'wide'), axis =1)
        initial_balance = initial_balance.drop(['narrow_ib','moderate_ib','wide_ib'], axis=1)

        initial_balance.rename(columns = {'high':'ib_high',
                                           'low':'ib_low',
                                           'width':'ib_width',
                                           'mid':'ib_mid',
                                           'range_open':'ib_range_open' ,
                                           'width_status': 'ib_width_status'}, inplace=True)
        #initial_balance = initial_balance.tail(1)


        return initial_balance

    except FileNotFoundError:
        pass

In [ ]:
def thread_ib(symbol):
    ib = compute_IB(symbol)
    ib.to_csv(f"IB/"+symbol+"ib.csv")

with concurrent.futures.ProcessPoolExecutor() as executor:
     ib = [executor.submit(thread_ib,symbol) 
               for symbol in tickers]

In [ ]:
file = glob.glob('IB/*.csv')
random_files=np.random.choice(file,2)
file = file[1]
random_df =  pd.read_csv(file)
ib_df = random_df.tail(1)

In [ ]:
for symbol in tickers:
    try:
        df = pd.read_csv(f"IB/"+symbol+"ib.csv")
        df = df.tail(1)
        ib_df = ib_df.append(df)
        ib_df = ib_df.drop_duplicates()
    except FileNotFoundError:
        pass

In [ ]:
ib_df.to_csv('IB/IB.csv', index=False)

In [ ]:
pd.read_csv('IB/IB.csv')

,date,symbol,ib_high,ib_low,ib_width,ib_mid,ib_range_open,opportunity,ib_width_status
0,2022-06-07,BLZUSDT,0.10220,0.09964,0.00256,0.100920,bottom,long,moderate
1,2022-06-07,IOTXUSDT,0.03491,0.03323,0.00168,0.034070,top,short,wide
2,2022-06-07,XLMUSDT,0.14513,0.14032,0.00481,0.142725,top,short,wide
3,2022-06-07,OMGUSDT,2.57600,2.47800,0.09800,2.527000,top,short,wide
4,2022-06-07,BALUSDT,7.60400,7.30600,0.29800,7.455000,top,short,wide
...,...,...,...,...,...,...,...,...,...
137,2022-06-07,API3USDT,1.57000,1.50500,0.06500,1.537500,top,short,wide
138,2022-06-07,TLMUSDT,0.03660,0.03530,0.00130,0.035950,bottom,long,moderate
139,2022-06-07,MATICUSDT,0.63490,0.60960,0.02530,0.622250,top,short,wide
140,2022-06-07,SNXUSDT,2.78000,2.67100,0.10900,2.725500,top,short,wide


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4b416ed7-4459-4831-a6c0-a85d5b5e69e0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>